In [1]:
def convert_snailfish_to_array(string):
    '''
    Wandle den String in ein Array um. Dabei werden die Klammern bewahrt, gleichzeitig Zahlen größer als 9 als einzelne Zahl abgespeichert
    '''
    integers = [str(i) for i in range(99)]
    arr = []
    i = 0
    while i < len(string):
        if string[i] == "[" or string[i] == "]": arr.append(string[i])
        if string[i] in integers:
            if string[i+1] in integers:
                arr.append(10*int(string[i])+int(string[i+1]))
                i+=1
            else:
                arr.append(int(string[i]))
        i+=1

    return arr

FWD = 1
BWD = 0
PAR_OPEN = "["
PAR_CLOSE = "]"
class DAY_18:
    def __init__(self, string, verbose=True):
        self.__snailfish_array = convert_snailfish_to_array(string)
        self.__verbose = verbose
        #if self.__verbose: print(self.__snailfish_array)
               
    def add_new_snailfish_string(self, new_string):
        new_array = convert_snailfish_to_array(new_string)
        new_snailfish = ["["]
        for element in self.__snailfish_array:
            new_snailfish.append(element)
        for element in new_array:
            new_snailfish.append(element)
        new_snailfish.append("]")
        self.__snailfish_array = new_snailfish
        #print(self.__snailfish_array)
        
    def convert_array_to_snailfish(self):
        converted = str(self.__snailfish_array[0])
        for i in range(1, len(self.__snailfish_array)):
            if self.__snailfish_array[i] == "[" and self.__snailfish_array[i-1] == "]":
                converted += ","+str(self.__snailfish_array[i])
            elif type(self.__snailfish_array[i]) == type(1) and \
                 type(self.__snailfish_array[i-1]) == type(1):
                converted += ","+str(self.__snailfish_array[i])
            elif type(self.__snailfish_array[i]) == type(1) and self.__snailfish_array[i-1] == "]":
                converted += ","+str(self.__snailfish_array[i])
            elif self.__snailfish_array[i] == "[" and type(self.__snailfish_array[i-1]) == type(1):
                converted += ","+str(self.__snailfish_array[i])                                
            else:
                converted += str(self.__snailfish_array[i])
        return converted
        
    def __search_next_integer(self, cur_pos, direction):
        '''
        Sucht ab der angegebenen Position nach dem nächstgelegenen Integer
        '''
        if cur_pos >= len(self.__snailfish_array): return None, None
        offset =  1#Gehe eine Position von der aktuellen weg um die nächste Zahl zu finden
        if direction == 1:#Vorwärts
            for i in range(cur_pos+offset, len(self.__snailfish_array)):
                if type(self.__snailfish_array[i]) == type(1):
                    return self.__snailfish_array[i], i

        else:#Rückwärts
            for i in range(cur_pos-offset, 0, -1):
                if type(self.__snailfish_array[i]) == type(1): 
                    return self.__snailfish_array[i], i
        return None, None
    
    def __replace(self, replace_interval, to_insert):
        '''
        Ersetzt die Listenelemente im gegebenen Intervall durch andere Elemente
        '''
        if len(replace_interval) == 1:
            del self.__snailfish_array[replace_interval[0]]
        else:
            del self.__snailfish_array[replace_interval[0]:replace_interval[-1]+1]
        for element in to_insert[::-1]:
            self.__snailfish_array.insert(replace_interval[0], element)
        return self.__snailfish_array
    
    def __explode(self, cur_pos):
        if self.__verbose: print("Explode", cur_pos)
        number_prev, index_prev = self.__search_next_integer(cur_pos, BWD)
        number_1, index_1 = self.__search_next_integer(cur_pos, FWD)
        if self.__verbose: print("prev", number_prev, index_prev)
        if self.__verbose: print("num_1", number_1, index_1)
        #Aktualisiere number_prev
        if index_prev != None:
            number_prev += number_1
            self.__replace([index_prev], [number_prev])
    
        number_2, index_2 = self.__search_next_integer(index_1, FWD)
        number_next, index_next = self.__search_next_integer(index_2, FWD)
        if self.__verbose: print("num_2", number_2, index_2)
        if self.__verbose: print("next", index_next, index_next)        
        #Aktualisiere number_next
        if index_next != None:
            number_next += number_2
            self.__replace([index_next], [number_next])
        if self.__verbose: print("after explode", self.__snailfish_array)   
        #Setze die Klammer auf 0
        if self.__verbose: print(index_1-1, index_2+1)
        if self.__verbose: print(self.__snailfish_array[index_2+1])
        #index_1-1 weil index_1 die erste Zahl ist und -1 die Klammer, welche beide ersetzt werden sollen
        #index_2+2 weil index_2 die zweite Zahl ist und +1 die Klammer
        self.__replace([index_1-1, index_2+1], [0])
        
    def __split(self, cur_pos):
        #if self.__verbose: print("Split", cur_pos)
        number_1 = self.__snailfish_array[cur_pos]//2
        number_2 = self.__snailfish_array[cur_pos]//2 + self.__snailfish_array[cur_pos]%2
        #if self.__verbose: print(number_1, number_2)
        self.__replace([cur_pos], ["[", number_1, number_2, "]"])
    
    def reduce(self):
        i = 0
        brackets = 0
        through = False
        exploded = False
        while i < len(self.__snailfish_array):
            #Zähle die geöffneten Klammern
            if self.__snailfish_array[i] == PAR_OPEN: 
                brackets += 1
            elif self.__snailfish_array[i] == PAR_CLOSE:
                brackets -= 1
                
            if brackets == 5:
                #print("Vor explode:", self.convert_array_to_snailfish())#self.__snailfish_array)
                self.__explode(i)
                #print("Nach explode:" , self.convert_array_to_snailfish())#self.__snailfish_array)
                #print("\n")
                i = 0
                brackets = 0
                exploded = True
                through = False
                #break
                continue
            if type(self.__snailfish_array[i]) == type(1) and self.__snailfish_array[i] > 9 and through:
                #print("Vor split: ", self.convert_array_to_snailfish())
                self.__split(i)
                #print("Nach split: ", self.convert_array_to_snailfish())
                #print("\n")
                i = 0
                brackets = 0
                #return
                continue
            i += 1
            if i == len(self.__snailfish_array) and exploded: 
                through = True
                exploded = False
                i = 0
            
    def get_snailfish_array(self):
        return self.convert_array_to_snailfish()
    
    def magnitude(self):
        i = 0
        brackets = 0
        while i < len(self.__snailfish_array)-3:
             #Zähle die geöffneten Klammern
            i+=1
            if self.__snailfish_array[i] == "[" and \
               type(self.__snailfish_array[i+1]) == type(1) and \
               type(self.__snailfish_array[i+2]) == type(1) and \
               self.__snailfish_array[i+3] == "]":
                self.__replace([i, i+3], [3*self.__snailfish_array[i+1]+2*self.__snailfish_array[i+2]])
                #print(self.__snailfish_array)
                i = 0
        return 3*self.__snailfish_array[1]+2*self.__snailfish_array[2]
        
with open("example_1.txt", "r") as f:
    cont = f.readlines()
    
day18_test = DAY_18(cont[0].rstrip(), False)
for line in cont[1:]:
    day18_test.add_new_snailfish_string(line.rstrip())
    day18_test.reduce()
print(day18_test.get_snailfish_array() == "[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]")
day18_test.magnitude()

with open("example_2.txt", "r") as f:
    cont = f.readlines()
day18_test = DAY_18(cont[0].rstrip(), False)
for line in cont[1:]:
    day18_test.add_new_snailfish_string(line.rstrip())
    day18_test.reduce()
print(day18_test.get_snailfish_array() == "[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]")
day18_test.magnitude()
        
with open("input.txt", "r") as f:
    cont = f.readlines()
#print(cont[0])
day18_puzzle = DAY_18(cont[0].rstrip(), False)
for line in cont[1:]:
    day18_puzzle.add_new_snailfish_string(line.rstrip())
    day18_puzzle.reduce()
print(day18_puzzle.get_snailfish_array())
day18_puzzle.magnitude()

True
True
[[[[7,7],[7,8]],[[7,7],[9,7]]],[[[7,9],[0,9]],[[8,9],[7,0]]]]


4323

In [2]:
with open("input.txt", "r") as f:
    cont = f.readlines()
magn = []
for string_1 in cont:
    for string_2 in cont:
        day18_puzzle_2 = DAY_18(string_1.rstrip(), False)
        day18_puzzle_2.add_new_snailfish_string(string_2.rstrip())
        day18_puzzle_2.reduce()
        magn.append(day18_puzzle_2.magnitude())
print(max(magn))

4749
